In [3]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [4]:
import numpy as np
import sys
import json
import os
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
from termcolor import colored

plt.rcParams["figure.figsize"] = (10,10)

# change this property
NOMEROFF_NET_DIR = os.path.abspath('/var/www/nomeroff-net')
sys.path.append(NOMEROFF_NET_DIR)

In [5]:
from NomeroffNet.BBoxNpPoints import (NpPointsCraft, 
                                      getCvZoneRGB, 
                                      convertCvZonesRGBtoBGR, 
                                      reshapePoints)
from NomeroffNet.YoloV5Detector import Detector
from NomeroffNet.OptionsDetector import OptionsDetector
from NomeroffNet.TextDetector import TextDetector

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

2021-04-22 10:31:09,547 [328017] WARNING  py.warnings:109: [JupyterRequire] /usr/local/lib64/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0



Loading weights of refiner from checkpoint (/mnt/data/var/www/nomeroff-net/NomeroffNet/Base/mcm/./models/NpPointsCraft/refiner/craft_refiner_CTW1500_2020-02-16.pth )


In [14]:
def load_dataset(json_path):
    dataset = {}
    with open(json_path) as json_file:
        data = json.load(json_file)
        for p in data['_via_img_metadata']:
            item = data['_via_img_metadata'][p]
            bboxes = []
            for region in item['regions']:
                x1 = min(region['shape_attributes']['all_points_x'])
                x2 = max(region['shape_attributes']['all_points_x'])
                y1 = min(region['shape_attributes']['all_points_y'])
                y2 = max(region['shape_attributes']['all_points_y'])
                bboxes.append({
                    'x1': x1, 
                    'x2': x2, 
                    'y1': y1, 
                    'y2': y2, 
                    'region_name': region['region_attributes']["region_name"].strip(),
                    'numberplate': region['region_attributes']["np"].strip(),
                })
            dataset[p] = bboxes
    return dataset

{'195715304.jpeg157772': [{'x1': 1,
   'x2': 167,
   'y1': 699,
   'y2': 826,
   'region_name': 'su',
   'numberplate': '0395ВНБ'}],
 '205676925.jpeg139880': [{'x1': 175,
   'x2': 236,
   'y1': 171,
   'y2': 224,
   'region_name': 'su',
   'numberplate': '7733КДВ'}],
 '213652557.jpeg82103': [{'x1': 196,
   'x2': 288,
   'y1': 556,
   'y2': 636,
   'region_name': 'eu_ua_2004',
   'numberplate': 'АААА0157'}],
 '235150655.jpeg87309': [{'x1': 302,
   'x2': 385,
   'y1': 315,
   'y2': 358,
   'region_name': 'eu_ua_2004',
   'numberplate': 'АВМІ0220'}],
 '242650861.jpeg113061': [{'x1': 211,
   'x2': 345,
   'y1': 522,
   'y2': 635,
   'region_name': 'su',
   'numberplate': '3940ОДК'}],
 '263370707.jpeg134413': [{'x1': 197,
   'x2': 268,
   'y1': 246,
   'y2': 303,
   'region_name': 'xx-transit',
   'numberplate': '115632АС'}],
 '274878190.jpeg129381': [{'x1': 104,
   'x2': 205,
   'y1': 631,
   'y2': 734,
   'region_name': 'eu_ua_2015',
   'numberplate': 'АВАВ0893'}],
 '276184165.jpeg107245'

In [14]:

def compare(photo_dir = os.path.join(NOMEROFF_NET_DIR, 'dataset/np/'),
            json_path = os.path.join(NOMEROFF_NET_DIR, 'dataset/np/via.json'),
            acc_less_than = 0.7):
    
    # init models
    npPointsCraft = NpPointsCraft()
    npPointsCraft.load()
    detector = Detector()
    detector.load()
    optionsDetector = OptionsDetector()
    optionsDetector.load()
    textDetector = TextDetector({
        "eu_ua_2004_2015": {
            "for_regions": ["eu_ua_2015", "eu_ua_2004"],
            "model_path": "latest"
        },
        "eu_ua_1995": {
            "for_regions": ["eu_ua_1995"],
            "model_path": "latest"
        },
        "eu": {
            "for_regions": ["eu"],
            "model_path": "latest"
        },
        "ru": {
            "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
            "model_path": "latest" 
        },
        "kz": {
            "for_regions": ["kz"],
            "model_path": "latest"
        },
        "ge": {
            "for_regions": ["ge"],
            "model_path": "latest"
        },
        "su": {
            "for_regions": ["su"],
            "model_path": "latest"
        }
    })
    dataset = load_dataset(json_path)
    

In [8]:
compare(
    json_path="/home/dmitroprobachay/Downloads/many_line_1new.json"
)

NameError: name 'json_path' is not defined